<a href="https://colab.research.google.com/github/shiwkumar/UAV_CR/blob/main/UAV_CR_Sample_Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy scipy h5py tqdm

In [5]:

import numpy as np
from scipy import signal
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Define parameters
signal_params = {
    'center_frequency': 2.4e9,  # Hz (2.4 GHz ISM band)
    'bandwidth': 20e6,         # Hz (20 MHz channels)
    'sampling_rate': 40e6,     # Hz (2x Nyquist rate)
    'duration': 1e-3,          # seconds per sample
    'snr_range': [-20, 5],     # dB
    'modulation_types': ['BPSK', 'QPSK', '16-QAM', '64-QAM']
}

uav_params = {
    'altitude_range': [50, 200],    # meters
    'velocity_range': [0, 15],      # m/s
    'number_of_uavs': [2, 4, 6, 8], # Different scenarios
}

channel_params = {
    'path_loss_exponent': 2.7,      # Urban environment
    'shadowing_std': 4.0,           # dB
    'fading_type': 'Rician',        # For LOS scenarios
    'K_factor': 10,                 # Rician K-factor
    'doppler_shift': True,          # Consider UAV mobility
}

training_data = {
    'size': 10000,            # Number of samples
    'distribution': {
        'training': 0.7,       # 70% for training
        'validation': 0.15,    # 15% for validation
        'testing': 0.15        # 15% for testing
    }
}

def generate_dataset():
    """Generate synthetic dataset for CR-UAV testing"""

    dataset = []

    for i in tqdm(range(training_data['size']), desc="Generating samples"):
        # Generate UAV position
        x = np.random.uniform(-100, 100)
        y = np.random.uniform(-100, 100)
        z = np.random.uniform(*uav_params['altitude_range'])
        position = (x, y, z)

        # Generate UAV velocity
        vx = np.random.uniform(-uav_params['velocity_range'][1], uav_params['velocity_range'][1])
        vy = np.random.uniform(-uav_params['velocity_range'][1], uav_params['velocity_range'][1])
        vz = np.random.uniform(-5, 5)  # Smaller vertical velocity range
        velocity = (vx, vy, vz)

        # Generate SNR
        snr = np.random.uniform(*signal_params['snr_range'])

        # Calculate signal power based on path loss and shadowing
        distance = np.sqrt(x**2 + y**2 + z**2)
        path_loss = 20 * np.log10(distance) + 20 * np.log10(signal_params['center_frequency']) - 147.55
        shadowing = np.random.normal(0, channel_params['shadowing_std'])

        # Calculate received power
        tx_power = 23  # dBm (typical WiFi transmit power)
        rx_power = tx_power - path_loss + shadowing

        # Generate channel conditions
        K = channel_params['K_factor']
        fading = np.sqrt(K/(K+1)) + np.sqrt(1/(K+1)) * (np.random.normal(0,1) + 1j*np.random.normal(0,1))
        fading_magnitude = np.abs(fading)

        # Generate spectrum occupancy (primary user presence)
        primary_user_present = 1 if np.random.random() > 0.5 else 0

        # Select modulation type
        modulation = np.random.choice(signal_params['modulation_types'])

        # Store sample
        sample = {
            'x': x,
            'y': y,
            'z': z,
            'vx': vx,
            'vy': vy,
            'vz': vz,
            'snr': snr,
            'path_loss': path_loss,
            'shadowing': shadowing,
            'rx_power': rx_power,
            'fading_magnitude': fading_magnitude,
            'primary_user_present': primary_user_present,
            'modulation': modulation
        }

        dataset.append(sample)

    return pd.DataFrame(dataset)

# Generate dataset
df = generate_dataset()

# Save to HDF5 file
df.to_hdf('cr_uav_dataset_10k.h5', key='data', mode='w')

# Print dataset statistics
print("\nDataset Statistics:")
print("-------------------")
print(f"Total samples: {len(df)}")
print(f"Primary user presence ratio: {df['primary_user_present'].mean():.2f}")
print("\nNumerical Features Statistics:")
print(df.describe())
print("\nModulation Distribution:")
print(df['modulation'].value_counts(normalize=True))

# Create visualization of key parameters
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# SNR Distribution
sns.histplot(data=df, x='snr', bins=30, ax=axes[0,0])
axes[0,0].set_title('SNR Distribution')

# Path Loss vs Distance
distance = np.sqrt(df['x']**2 + df['y']**2 + df['z']**2)
axes[0,1].scatter(distance, df['path_loss'], alpha=0.1)
axes[0,1].set_title('Path Loss vs Distance')
axes[0,1].set_xlabel('Distance (m)')
axes[0,1].set_ylabel('Path Loss (dB)')

# Received Power Distribution
sns.histplot(data=df, x='rx_power', bins=30, ax=axes[1,0])
axes[1,0].set_title('Received Power Distribution')

# UAV Altitude Distribution
sns.histplot(data=df, x='z', bins=30, ax=axes[1,1])
axes[1,1].set_title('UAV Altitude Distribution')

plt.tight_layout()
plt.savefig('dataset_visualization.png')
plt.close()

# Save dataset split indices
n_samples = len(df)
indices = np.random.permutation(n_samples)
train_size = int(n_samples * training_data['distribution']['training'])
val_size = int(n_samples * training_data['distribution']['validation'])

train_indices = indices[:train_size]
val_indices = indices[train_size:train_size+val_size]
test_indices = indices[train_size+val_size:]

np.savez('dataset_splits.npz',
         train_indices=train_indices,
         val_indices=val_indices,
         test_indices=test_indices)

print("\nDataset Split Information:")
print(f"Training samples: {len(train_indices)}")
print(f"Validation samples: {len(val_indices)}")
print(f"Testing samples: {len(test_indices)}")


Generating samples: 100%|██████████| 10000/10000 [00:00<00:00, 10210.97it/s]



Dataset Statistics:
-------------------
Total samples: 10000
Primary user presence ratio: 0.49

Numerical Features Statistics:
                  x             y             z            vx            vy  \
count  10000.000000  10000.000000  10000.000000  10000.000000  10000.000000   
mean      -0.204654      0.619572    125.446803     -0.074512     -0.025586   
std       57.639156     57.378128     42.884137      8.732563      8.661804   
min      -99.990328    -99.984903     50.004507    -14.999576    -14.998112   
25%      -50.216813    -49.583661     88.544538     -7.736886     -7.495069   
50%        0.717120      0.996584    125.880652     -0.112508     -0.150204   
75%       49.356557     50.159262    162.190944      7.529662      7.564317   
max       99.986033     99.976639    199.998335     14.997358     14.998199   

                 vz           snr     path_loss     shadowing      rx_power  \
count  10000.000000  10000.000000  10000.000000  10000.000000  10000.000000   
me

In [11]:
import numpy as np
import pandas as pd1
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from tqdm import tqdm
import h5py

class StackelbergGame:
    def __init__(self, num_secondary_users=4):
        self.num_su = num_secondary_users
        self.sensing_time = 0.001  # 1ms sensing time
        self.sampling_rate = 40e6  # 40MHz sampling rate
        self.samples_per_sensing = int(self.sensing_time * self.sampling_rate)
        self.noise_variance = 1.0
        self.price_coefficient = 0.1
        self.energy_coefficient = 0.01

    def leader_utility(self, price, detection_prob):
        """Primary user (leader) utility function"""
        return price * detection_prob - self.price_coefficient * price**2

    def follower_utility(self, detection_prob, sensing_time, price):
        """Secondary user (follower) utility function"""
        energy_cost = self.energy_coefficient * sensing_time
        return detection_prob - price * sensing_time - energy_cost

    def calculate_detection_probability(self, snr, sensing_time):
        """Calculate probability of detection using SNR and sensing time"""
        samples = int(sensing_time * self.sampling_rate)
        Q = lambda x: 0.5 * (1 - norm.cdf(x/np.sqrt(2)))
        pf = 0.1  # Fixed false alarm probability

        lambda_thresh = self.noise_variance * (1 + np.sqrt(2/samples) * norm.ppf(1-pf))
        gamma = 10**(snr/10)
        pd = Q((lambda_thresh/self.noise_variance - gamma - 1) * np.sqrt(samples/2/(2*gamma + 1)))
        return pd

    def optimize_price(self, snr):
        """Optimize price for primary user"""
        prices = np.linspace(0.1, 2, 100)
        max_utility = -np.inf
        optimal_price = prices[0]

        for price in prices:
            sensing_time = self.optimize_sensing_time(price, snr)
            pd = self.calculate_detection_probability(snr, sensing_time)
            utility = self.leader_utility(price, pd)

            if utility > max_utility:
                max_utility = utility
                optimal_price = price

        return optimal_price

    def optimize_sensing_time(self, price, snr):
        """Optimize sensing time for secondary users"""
        sensing_times = np.linspace(0.1e-3, 2e-3, 100)
        max_utility = -np.inf
        optimal_time = sensing_times[0]

        for time in sensing_times:
            pd = self.calculate_detection_probability(snr, time)
            utility = self.follower_utility(pd, time, price)

            if utility > max_utility:
                max_utility = utility
                optimal_time = time

        return optimal_time

def run_simulation(dataset_path, simulation_time=60):
    """Run spectrum sensing simulation for 60 seconds"""
    # Load dataset
    df = pd1.read_hdf(dataset_path)

    # Initialize parameters
    game = StackelbergGame(num_secondary_users=4)
    sensing_interval = 0.001  # 1ms
    num_iterations = int(simulation_time / sensing_interval)
    snr_range = np.arange(-20, 6, 1)

    # Initialize results storage
    detection_probabilities = {snr: [] for snr in snr_range}
    price_history = []
    sensing_time_history = []

    # Run simulation
    print("\nRunning Stackelberg Game simulation...")
    for snr in tqdm(snr_range):
        # Filter dataset for current SNR
        snr_data = df[np.abs(df['snr'] - snr) < 0.5]

        if len(snr_data) > 0:
            # Optimize price and sensing time
            optimal_price = game.optimize_price(snr)
            optimal_sensing_time = game.optimize_sensing_time(optimal_price, snr)

            # Calculate detection probability
            pd = game.calculate_detection_probability(snr, optimal_sensing_time)

            detection_probabilities[snr].append(pd)
            price_history.append(optimal_price)
            sensing_time_history.append(optimal_sensing_time)

    # Process results
    avg_detection_prob = {snr: np.mean(probs) if probs else 0
                         for snr, probs in detection_probabilities.items()}

    # Create visualization
    plt.figure(figsize=(12, 8))

    # Plot detection probability vs SNR
    snrs = list(avg_detection_prob.keys())
    pds = list(avg_detection_prob.values())

    plt.plot(snrs, pds, 'b-', linewidth=2, label='Stackelberg Game')

    # Add theoretical curve for comparison
    theoretical_pd = [game.calculate_detection_probability(snr, np.mean(sensing_time_history))
                     for snr in snrs]
    plt.plot(snrs, theoretical_pd, 'r--', linewidth=2, label='Theoretical')

    plt.grid(True)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Probability of Detection')
    plt.title('Spectrum Sensing Performance using Stackelberg Game\n'
              f'(Number of SUs: {game.num_su}, Simulation Time: {simulation_time}s)')
    plt.legend()
    plt.ylim([0, 1])

    # Save plot
    plt.savefig('stackelberg_simulation_results.png')
    plt.close()

    # Print results
    print("\nSimulation Results:")
    print("------------------")
    print(f"Average Detection Probability: {np.mean(list(avg_detection_prob.values())):.4f}")
    print(f"Average Price: {np.mean(price_history):.4f}")
    print(f"Average Sensing Time: {np.mean(sensing_time_history)*1000:.2f} ms")

    # Save detailed results
    results = {
        'snr': snrs,
        'detection_probability': pds,
        'theoretical_pd': theoretical_pd,
        'price_history': price_history,
        'sensing_time_history': sensing_time_history
    }

    return results

if __name__ == "__main__":
    # Run simulation
    results = run_simulation('cr_uav_dataset_10k.h5')

    # Save results to file
    np.savez('stackelberg_simulation_results.npz', **results)

    # Create additional analysis plots
    plt.figure(figsize=(15, 5))

    # Price history
    plt.subplot(121)
    plt.plot(results['snr'], results['price_history'], 'g-')
    plt.grid(True)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Optimal Price')
    plt.title('Price vs SNR')

    # Sensing time history
    plt.subplot(122)
    plt.plot(results['snr'], np.array(results['sensing_time_history'])*1000, 'm-')
    plt.grid(True)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Optimal Sensing Time (ms)')
    plt.title('Sensing Time vs SNR')

    plt.tight_layout()
    plt.savefig('stackelberg_additional_analysis.png')
    plt.close()


Running Stackelberg Game simulation...


100%|██████████| 26/26 [01:25<00:00,  3.31s/it]



Simulation Results:
------------------
Average Detection Probability: 0.4878
Average Price: 1.9897
Average Sensing Time: 0.63 ms


In [14]:
import numpy as np
import pandas as pd3
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
import seaborn as sns

class StackelbergGame:
    def __init__(self, num_secondary_users=4):
        self.num_sus = num_secondary_users
        self.sensing_time = 0.001  # 1ms sensing interval
        self.total_time = 60  # 1 minute simulation
        self.num_iterations = int(self.total_time / self.sensing_time)

        # Game parameters
        self.price_coefficient = 0.1
        self.cost_coefficient = 0.05
        self.interference_threshold = -80  # dBm
        self.energy_constraint = 100  # mJ

    def utility_primary(self, price, interference, revenue_coefficient=1.0):
        """Primary user utility function"""
        return revenue_coefficient * price - self.cost_coefficient * interference

    def utility_secondary(self, detection_prob, price, energy):
        """Secondary user utility function"""
        return np.log(1 + detection_prob) - price * energy

    def calculate_detection_probability(self, snr, sensing_time, pf=0.1):
        """Calculate probability of detection using SNR"""
        samples = int(sensing_time * 40e6)  # Using sampling rate from dataset
        gamma = 10**(snr/10)
        Q_inv = stats.norm.ppf(1-pf)

        pd = stats.norm.sf((Q_inv * np.sqrt(2*samples) - gamma * np.sqrt(samples)) /
                          np.sqrt(2*samples + 4*gamma*samples))
        return pd

    def optimize_price(self, snr, interference):
        """Primary user optimization"""
        prices = np.linspace(0.1, 1.0, 100)
        utilities = [self.utility_primary(p, interference) for p in prices]
        return prices[np.argmax(utilities)]

    def optimize_sensing(self, price, snr, energy_available):
        """Secondary user optimization"""
        sensing_times = np.linspace(0.1e-3, 1e-3, 100)
        utilities = []

        for st in sensing_times:
            pd = self.calculate_detection_probability(snr, st)
            energy = st * energy_available
            utility = self.utility_secondary(pd, price, energy)
            utilities.append(utility)

        return sensing_times[np.argmax(utilities)]

def run_simulation():
    # Load dataset
    df = pd3.read_hdf('cr_uav_dataset_10k.h5', key='data')

    # Initialize simulation parameters
    game = StackelbergGame(num_secondary_users=4)
    snr_range = np.arange(-20, 6, 1)
    results = []

    # Run simulation for each SNR value
    for snr in tqdm(snr_range, desc="Simulating SNR ranges"):
        snr_results = []

        # Filter dataset for current SNR range
        df_snr = df[np.abs(df['snr'] - snr) < 0.5]
        if len(df_snr) < game.num_iterations:
            df_snr = df_snr.sample(n=game.num_iterations, replace=True)

        # Run iterations for current SNR
        for i in range(game.num_iterations):
            # Get current channel conditions
            current_data = df_snr.iloc[i]
            interference = current_data['rx_power']
            true_presence = current_data['primary_user_present']

            # Primary user strategy
            optimal_price = game.optimize_price(snr, interference)

            # Secondary users strategies
            detection_probs = []
            for su in range(game.num_sus):
                # Each SU has slightly different energy availability
                energy_available = game.energy_constraint * (0.8 + 0.4 * np.random.random())
                optimal_sensing_time = game.optimize_sensing(optimal_price, snr, energy_available)
                pd = game.calculate_detection_probability(snr, optimal_sensing_time)
                detection_probs.append(pd)

            # Cooperative decision (OR-rule)
            final_detection = 1 - np.prod([1-pd for pd in detection_probs])
            snr_results.append({
                'snr': snr,
                'detection_prob': final_detection,
                'true_presence': true_presence,
                'price': optimal_price,
                'avg_sensing_time': np.mean(optimal_sensing_time)
            })

        results.extend(snr_results)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Calculate average detection probability for each SNR
    snr_performance = results_df.groupby('snr').agg({
        'detection_prob': ['mean', 'std'],
        'price': 'mean',
        'avg_sensing_time': 'mean'
    }).reset_index()

    # Plot results
    plt.figure(figsize=(12, 8))
    plt.errorbar(snr_performance['snr'],
                snr_performance['detection_prob']['mean'],
                yerr=snr_performance['detection_prob']['std'],
                fmt='o-', capsize=5)
    plt.grid(True)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Probability of Detection')
    plt.title('Detection Performance vs SNR in UAV Network\n(Stackelberg Game)')
    plt.savefig('stackelberg_simulation_results.png')
    plt.close()

    # Create detailed performance report
    print("\nSimulation Results Summary")
    print("--------------------------")
    print(f"Number of Secondary Users: {game.num_sus}")
    print(f"Total Simulation Time: {game.total_time} seconds")
    print(f"Sensing Interval: {game.sensing_time} seconds")
    print("\nPerformance Metrics:")
    print(snr_performance.round(4))

    # Save results
    results_df.to_csv('stackelberg_simulation_results.csv', index=False)

    return results_df, snr_performance

# Additional visualization for game dynamics
def plot_game_dynamics(results_df):
    plt.figure(figsize=(15, 5))

    # Plot 1: Price vs SNR
    plt.subplot(1, 3, 1)
    sns.boxplot(x='snr', y='price', data=results_df)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Optimal Price')
    plt.title('Price Dynamics')

    # Plot 2: Sensing Time vs SNR
    plt.subplot(1, 3, 2)
    sns.boxplot(x='snr', y='avg_sensing_time', data=results_df)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Average Sensing Time (s)')
    plt.title('Sensing Time Dynamics')

    # Plot 3: Detection Probability Distribution
    plt.subplot(1, 3, 3)
    sns.boxplot(x='snr', y='detection_prob', data=results_df)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Detection Probability')
    plt.title('Detection Probability Distribution')

    plt.tight_layout()
    plt.savefig('stackelberg_game_dynamics.png')
    plt.close()

if __name__ == "__main__":
    # Run simulation
    results_df, snr_performance = run_simulation()

    # Plot additional dynamics
    plot_game_dynamics(results_df)

Simulating SNR ranges:   0%|          | 0/26 [01:32<?, ?it/s]


KeyboardInterrupt: 

In [16]:
pip install keras

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc

class CNNSpectrumSensor:
    def __init__(self, num_secondary_users=4):
        self.num_sus = num_secondary_users
        self.sensing_time = 0.001  # 1ms sensing interval
        self.total_time = 60      # 1 minute simulation
        self.num_iterations = int(self.total_time / self.sensing_time)
        self.window_size = 128    # Input window size for CNN
        self.model = None

    def build_cnn_model(self):
        """Build CNN model for spectrum sensing"""
        model = models.Sequential([
            # Input layer
            layers.Input(shape=(self.window_size, 6)),  # [I, Q, SNR, path_loss, fading, position]

            # Convolutional layers
            layers.Conv1D(64, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling1D(2),

            layers.Conv1D(128, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling1D(2),

            layers.Conv1D(256, 3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling1D(2),

            # Dense layers
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

        return model

    def prepare_data(self, df):
        """Prepare data for CNN"""
        # Create feature matrix
        X = np.zeros((len(df), self.window_size, 6))

        for i in range(len(df)):
            # Generate I/Q samples based on SNR and channel conditions
            t = np.linspace(0, self.sensing_time, self.window_size)
            snr = df.iloc[i]['snr']
            path_loss = df.iloc[i]['path_loss']
            fading = df.iloc[i]['fading_magnitude']

            # Generate complex signal
            signal = np.exp(1j * 2 * np.pi * 1000 * t)  # 1 kHz signal
            signal = signal * np.sqrt(10**(snr/10))

            # Add channel effects
            signal = signal * fading * 10**(-path_loss/20)

            # Add noise
            noise = np.random.normal(0, 1, self.window_size) + 1j * np.random.normal(0, 1, self.window_size)
            noisy_signal = signal + noise

            # Fill feature matrix
            X[i, :, 0] = noisy_signal.real  # I component
            X[i, :, 1] = noisy_signal.imag  # Q component
            X[i, :, 2] = snr * np.ones(self.window_size)  # SNR
            X[i, :, 3] = path_loss * np.ones(self.window_size)  # Path loss
            X[i, :, 4] = fading * np.ones(self.window_size)  # Fading
            X[i, :, 5] = np.sqrt(df.iloc[i]['x']**2 + df.iloc[i]['y']**2 + df.iloc[i]['z']**2)  # Distance

        # Normalize features
        for i in range(X.shape[2]):
            scaler = StandardScaler()
            X[:, :, i] = scaler.fit_transform(X[:, :, i])

        y = df['primary_user_present'].values
        return X, y

def run_simulation():
    try:
        # Load dataset
        print("Loading dataset...")
        df = pd.read_hdf('cr_uav_dataset_10k.h5', key='data')

        # Initialize simulation parameters
        cnn_sensor = CNNSpectrumSensor(num_secondary_users=4)

        # Create SNR bins with wider ranges to ensure sufficient samples
        snr_bins = np.arange(-20, 6, 2)  # Changed step size to 2
        results = []

        # Print dataset statistics
        print(f"Total samples in dataset: {len(df)}")
        print(f"SNR range in dataset: [{df['snr'].min():.1f}, {df['snr'].max():.1f}]")

        # Run simulation for each SNR bin
        for snr_min, snr_max in zip(snr_bins[:-1], snr_bins[1:]):
            try:
                # Filter dataset for current SNR range
                df_snr = df[(df['snr'] >= snr_min) & (df['snr'] < snr_max)]

                print(f"Processing SNR range [{snr_min}, {snr_max}] dB with {len(df_snr)} samples")

                if len(df_snr) < 100:  # Reduced minimum sample requirement
                    print(f"Insufficient samples for SNR range [{snr_min}, {snr_max}] dB")
                    continue

                # Prepare data
                X, y = cnn_sensor.prepare_data(df_snr)

                # Split data
                split_idx = int(0.8 * len(X))
                X_train, X_test = X[:split_idx], X[split_idx:]
                y_train, y_test = y[:split_idx], y[split_idx:]

                # Build and train model
                cnn_sensor.model = cnn_sensor.build_cnn_model()

                # Train model
                history = cnn_sensor.model.fit(
                    X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=0
                )

                # Evaluate performance
                y_pred = cnn_sensor.model.predict(X_test, verbose=0)
                fpr, tpr, _ = roc_curve(y_test, y_pred)

                # Calculate cooperative detection (OR-rule for multiple SUs)
                coop_detection = 1 - np.prod([1-tpr[1] for _ in range(cnn_sensor.num_sus)])

                # Store results using average SNR of the bin
                avg_snr = (snr_min + snr_max) / 2
                results.append({
                    'snr': avg_snr,
                    'snr_min': snr_min,
                    'snr_max': snr_max,
                    'num_samples': len(df_snr),
                    'detection_prob': coop_detection,
                    'false_alarm_prob': fpr[1],
                    'training_accuracy': history.history['accuracy'][-1],
                    'validation_accuracy': history.history['val_accuracy'][-1]
                })

            except Exception as e:
                print(f"Error processing SNR range [{snr_min}, {snr_max}] dB: {str(e)}")
                continue

        if not results:
            raise ValueError("No valid results generated from any SNR range")

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)

        # Sort results by SNR
        results_df = results_df.sort_values('snr')

        # Create plots
        plt.figure(figsize=(12, 8))
        plt.errorbar(results_df['snr'], results_df['detection_prob'],
                    xerr=(results_df['snr_max'] - results_df['snr_min'])/2,
                    fmt='o-', label='Cooperative Detection',
                    linewidth=2, markersize=8, capsize=5)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('SNR (dB)', fontsize=12)
        plt.ylabel('Probability of Detection', fontsize=12)
        plt.title(f'CNN-based Detection Performance vs SNR\n({cnn_sensor.num_sus} Secondary Users)',
                  fontsize=14, pad=20)
        plt.legend(fontsize=10)
        plt.xlim(min(results_df['snr'])-2, max(results_df['snr'])+2)
        plt.ylim(0, 1.1)
        plt.grid(True)
        plt.savefig('cnn_simulation_results.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Plot training metrics
        plt.figure(figsize=(15, 5))

        # Training/Validation Accuracy
        plt.subplot(1, 3, 1)
        plt.plot(results_df['snr'], results_df['training_accuracy'], 'o-',
                label='Training', linewidth=2, markersize=6)
        plt.plot(results_df['snr'], results_df['validation_accuracy'], 's-',
                label='Validation', linewidth=2, markersize=6)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('SNR (dB)', fontsize=12)
        plt.ylabel('Accuracy', fontsize=12)
        plt.title('Model Accuracy vs SNR', fontsize=14)
        plt.legend(fontsize=10)
        plt.ylim(0, 1.1)

        # Sample Distribution
        plt.subplot(1, 3, 2)
        plt.bar(results_df['snr'], results_df['num_samples'])
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('SNR (dB)', fontsize=12)
        plt.ylabel('Number of Samples', fontsize=12)
        plt.title('Sample Distribution', fontsize=14)

        # ROC-like plot
        plt.subplot(1, 3, 3)
        plt.plot(results_df['false_alarm_prob'], results_df['detection_prob'], 'o-',
                linewidth=2, markersize=6, color='green')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xlabel('Probability of False Alarm', fontsize=12)
        plt.ylabel('Probability of Detection', fontsize=12)
        plt.title('Detection vs False Alarm', fontsize=14)
        plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
        plt.axis('square')
        plt.xlim(0, 1)
        plt.ylim(0, 1)

        plt.tight_layout()
        plt.savefig('cnn_training_metrics.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Print performance summary
        print("\nCNN Simulation Results Summary")
        print("-----------------------------")
        print(f"Number of Secondary Users: {cnn_sensor.num_sus}")
        print(f"Total Simulation Time: {cnn_sensor.total_time} seconds")
        print(f"Sensing Interval: {cnn_sensor.sensing_time} seconds")
        print("\nPerformance Metrics:")
        print(results_df.round(4))

        # Save results
        results_df.to_csv('cnn_simulation_results.csv', index=False)

        return results_df

    except Exception as e:
        print(f"Simulation failed: {str(e)}")
        return None

if __name__ == "__main__":
    # Set random seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    # Run simulation
    results_df = run_simulation()

Loading dataset...
Total samples in dataset: 10000
SNR range in dataset: [-20.0, 5.0]
Processing SNR range [-20, -18] dB with 801 samples
Processing SNR range [-18, -16] dB with 808 samples
Processing SNR range [-16, -14] dB with 745 samples


Processing SNR range [-14, -12] dB with 844 samples


Processing SNR range [-12, -10] dB with 800 samples
Processing SNR range [-10, -8] dB with 835 samples
Processing SNR range [-8, -6] dB with 779 samples
Processing SNR range [-6, -4] dB with 787 samples
Processing SNR range [-4, -2] dB with 750 samples
Processing SNR range [-2, 0] dB with 813 samples
Processing SNR range [0, 2] dB with 817 samples
Processing SNR range [2, 4] dB with 832 samples

CNN Simulation Results Summary
-----------------------------
Number of Secondary Users: 4
Total Simulation Time: 60 seconds
Sensing Interval: 0.001 seconds

Performance Metrics:
     snr  snr_min  snr_max  num_samples  detection_prob  false_alarm_prob  \
0  -19.0      -20      -18          801          0.0000            0.0133   
1  -17.0      -18      -16          808          0.0000            0.0122   
2  -15.0      -16      -14          745          0.0503            0.0000   
3  -13.0      -14      -12          844          0.0523            0.0000   
4  -11.0      -12      -10          80